In [ ]:
import os
os.environ['SPS_HOME'] = '/home/annalena/sps_fsps'

# Load SSP template

In [ ]:
# NBVAL_SKIP
from rubix.spectra.ssp.grid import HDF5SSPGrid
from rubix.utils import get_config

config = get_config("../rubix/config/rubix_config.yml")

ssp_bc = HDF5SSPGrid.from_file(config["ssp"]["templates"]["BruzualCharlot2003"], file_location="../rubix/spectra/ssp/templates")

In [ ]:
age_values = ssp_bc.age
print(age_values)

metallicity_values = ssp_bc.metallicity
print(metallicity_values)

# Load pipeline

In [ ]:
from rubix.core.pipeline import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": True,
            "subset_size": 2,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "BruzualCharlot2003"
        },
    },        
}
pipe = RubixPipeline(config)
rubixdata = pipe.run()

# Set target values

In [ ]:
import jax.numpy as jnp

rubixdata.stars.age = jnp.array([age_values[100], age_values[100]])
rubixdata.stars.metallicity = jnp.array([metallicity_values[3], metallicity_values[3]])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [ ]:
from rubix.core.pipeline_gradient import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
     "data": {
        "args": {
            "particle_type": ["stars"],
        },
    },
    
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "BruzualCharlot2003"
        },
    },        
}
pipe = RubixPipeline(config)
rubixdata = pipe.run(rubixdata)

target = rubixdata

# Calculate gradient value

In [ ]:
age_values[100]

In [ ]:
age_values[120]

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

# Initialize the pipeline
pipe = RubixPipeline(config)

age_test = jnp.array([[age_values[120], age_values[120]]])
loss = pipe.loss_only_wrt_age(age_test, rubixdata, target)
grad_stars_age120 = jax.grad(pipe.loss_only_wrt_age, argnums=0)(age_test, rubixdata, target)

In [ ]:
print(grad_stars_age120)

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

# Initialize the pipeline
pipe = RubixPipeline(config)

# Prepare the data structure to store results
results = []

#age_variations = [-1.0,-0.5,-0.2,-0.1,-0.05,-0.02,-0.01,-0.005,-0.002,-0.001,0.0,0.001,0.002,0.005,0.01,0.02,0.05,0.1,0.2,0.5,1.0]
age_variations = jnp.arange(-0.001, 0.001, 1e-5)
# Iterate over the age values from 1.0 to 10.3 in 0.1 steps
for age in age_variations:
    age_test = jnp.array([[age_values[120]+age, age_values[120]+age]])
    #print(metallicity_test)
    loss = pipe.loss_only_wrt_age(age_test, rubixdata, target)
    results.append((age, age_test, loss))

In [ ]:
# Save the results to a file
with open(f'./output/loss_grid/finite_difference3.txt', 'w') as f:
    for age_variation, age, loss in results:
        f.write(f"Age variation: {age_variation}, Age: {age}, Loss: {loss}\n")